In [5]:
#간단 테스팅용입니다
#json파일로 만들거는 나중에 age_gender_predicted_json.ipynb 형태로 할 예정

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet50
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import json
from sklearn.metrics import mean_absolute_error, accuracy_score


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
print(torch.__version__)
print(torch.cuda.is_available())  # True면 GPU 인식됨

2.2.2+cu121
True


In [ ]:
class DataProcessing(Dataset):
    '''
    변수 정리

    image_dir :이미지 디렉토리
    label_dir : 라벨 디렉토리 (json형태 리스트형)
    transform : 전처리 및 정규화 (if 문으로 만약 전처리 진행한다면 실행하게..)
    categories : 폴더 감정 4개 정의 되어있는 카테고리 뜻함
    emotion_return : 감정분석까지 할거면 활용할것
    samples : 빈리스트 (채워지는 곳임)

    '''

    def __init__(self, image_dir, label_dir, categories, transform=None, emotion_return=False, mode='train') :
        self.samples=[]
        self.transform = transform
        self.label_map= { cat : idx for idx, cat in enumerate(categories)}
        self.emotion_return = emotion_return
        self.mode = mode
        
        # 나이 정규화 위한 최소/최대 나이 정의 (예: 20~60)
        self.age_min = 10
        self.age_max = 60

        for category in categories :
            json_path =os.path.join(label_dir, f'{self.mode}_{category}.json')
            img_folder = os.path.join(image_dir,category)

            with open(json_path,'r',encoding='euc-kr') as f:
                label_data = json.load(f)

            for row in label_data :
                filename = row['filename']

                img_path = os.path.join(img_folder,filename)

                if os.path.isfile(img_path) :
                    
                    age = row.get('age')
                    
                    # 나이 정규화 (0~1 사이)
                    if age is not None:
                        age_norm = (age - self.age_min) / (self.age_max - self.age_min)
                    else:
                        age_norm = 0.0  # 결측값 처리 예시
                        
                    sample = {
                        'img_path' : img_path,
                        'category' : category,
                        'age' : age_norm,
                        'gender' : row.get('gender')
                    }
                    self.samples.append(sample)


    def __len__(self) :
        return len(self.samples)


    def __getitem__(self,idx) :
        sample = self.samples[idx]
        image = Image.open(sample['img_path']).convert('RGB')

        if self.transform :
            image = self.transform(image)


        #emotion_label = torch.tensor(self.label_map[sample['category']], dtype=torch.long) 수정1
        age=torch.tensor(sample['age'], dtype=torch.float32)
        gender = torch.tensor(1 if sample['gender']=='남' else 0, dtype=torch.long)

        # return image, emotion_label, age, gender 수정1

        if self.emotion_return :
            emotion_label = torch.tensor(self.label_map[sample['category']], dtype=torch.long)
            return image, emotion_label, age, gender

        else :
            return image, age, gender

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [27]:
model = resnet50()
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features,256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.4),
    
    nn.Linear(256,128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.3),
    
    nn.Linear(128,64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(0.2),
    
    nn.Linear(64,3)  # [age(1), gender(2)] 출력
)

In [ ]:

base_dir = os.path.dirname(os.path.abspath(__file__))
weight_path = os.path.join(base_dir, 'pth_pkl', 'model_raw_weights_v1_2_1_1.pth')

model.load_state_dict(torch.load(weight_path, map_location=device))
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
categories = ['anger','happy','panic','sadness']
base_dir = os.path.dirname(os.path.abspath(__file__))
test_image_dir = os.path.join(base_dir, 'test', 'image')
test_label_dir = os.path.join(base_dir, 'test', 'label')

In [30]:
test_dataset = DataProcessing(test_image_dir, test_label_dir, categories,
                              transform=transform, emotion_return=False, mode='test')
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [31]:
# 누적할 리스트 생성
all_true_ages = []
all_pred_ages = []
all_true_genders = []
all_pred_genders = []

# 5. 추론 + 평가
with torch.no_grad():
    for images, ages, genders in test_loader:
        images = images.to(device)
        ages = ages.to(device)
        genders = genders.to(device)

        outputs = model(images)
        pred_age = outputs[:,0]
        pred_gender_logits = outputs[:,1:3]

        # 나이 역정규화
        pred_age_orig = pred_age * (test_dataset.age_max - test_dataset.age_min) + test_dataset.age_min
        true_age_orig = ages * (test_dataset.age_max - test_dataset.age_min) + test_dataset.age_min

        pred_gender = torch.argmax(pred_gender_logits, dim=1)

        # 리스트에 누적
        all_pred_ages.extend(pred_age_orig.cpu().numpy())
        all_true_ages.extend(true_age_orig.cpu().numpy())
        all_pred_genders.extend(pred_gender.cpu().numpy())
        all_true_genders.extend(genders.cpu().numpy())

        # 샘플 출력
        for i in range(len(images)):
            print(f'predicted age : {pred_age_orig[i].item():.2f}세, actual age: {true_age_orig[i].item():.2f}세')
            print(f'predicted gender: {'남' if pred_gender[i].item()==1 else '여'}, actual gender: {'남' if genders[i].item()==1 else '여'}')

# 전체 결과 계산
test_mae = mean_absolute_error(all_true_ages, all_pred_ages)
test_acc = accuracy_score(all_true_genders, all_pred_genders)

print('\n 테스트 평가 결과')
print(f'나이 MAE: {test_mae:.4f}')
print(f'성별 정확도: {test_acc:.4f}')

predicted age : 27.16세, actual age: 30.00세
predicted gender: 여, actual gender: 여
predicted age : 27.11세, actual age: 40.00세
predicted gender: 남, actual gender: 남
predicted age : 24.28세, actual age: 30.00세
predicted gender: 남, actual gender: 남
predicted age : 27.19세, actual age: 20.00세
predicted gender: 남, actual gender: 남
predicted age : 29.78세, actual age: 30.00세
predicted gender: 여, actual gender: 여
predicted age : 23.93세, actual age: 20.00세
predicted gender: 남, actual gender: 남
predicted age : 24.73세, actual age: 20.00세
predicted gender: 여, actual gender: 여
predicted age : 24.53세, actual age: 40.00세
predicted gender: 남, actual gender: 남
predicted age : 27.58세, actual age: 30.00세
predicted gender: 여, actual gender: 여
predicted age : 28.00세, actual age: 20.00세
predicted gender: 여, actual gender: 여
predicted age : 24.83세, actual age: 40.00세
predicted gender: 남, actual gender: 남
predicted age : 27.27세, actual age: 20.00세
predicted gender: 여, actual gender: 여
predicted age : 27.19세, actu

In [33]:
import pandas as pd

In [ ]:
# 성별 숫자 → 문자 변환 함수
def gender_to_str(gender_int):
    return '남' if gender_int == 1 else '여'

# DataFrame 만들기
import numpy as np
df = pd.DataFrame({
    '예측 나이': np.round(all_pred_ages, 2),
    '실제 나이': np.round(all_true_ages, 2),
    '예측 성별': [gender_to_str(g) for g in all_pred_genders],
    '실제 성별': [gender_to_str(g) for g in all_true_genders]
})

# CSV 저장 (utf-8-sig로 한글 깨짐 방지)
base_dir = os.path.dirname(os.path.abspath(__file__))
save_path = os.path.join(base_dir, 'test_result', 'test_v1_2_1_1.csv')

df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"CSV 파일로 결과 저장 완료: {save_path}")

CSV 파일로 결과 저장 완료: test_v1_2_1_1.csv
